# Análisis Estadístico de Tesis: Evolución de la Calidad del Código

**Autor:** Investigador Principal
**Asistente de Investigación:** Modelo de IA
**Fecha:** 2 de agosto de 2025

Este notebook contiene el análisis de datos completo para la tesis de grado sobre la evolución de la calidad del código en estudiantes de Ingeniería en Sistemas.

## Paso 0: Configuración e Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, wilcoxon

# Configuración de estilo para los gráficos
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## Tarea 1: Preparación y Normalización de Datos

In [ ]:
# --- Carga de Datos ---
# IMPORTANTE: Sube tu archivo 'estudiantes_con_metricas_sonarcloud_20250802_194306.csv' al entorno de Colab antes de ejecutar esta celda.
file_path = 'estudiantes_con_metricas_sonarcloud_20250802_194306.csv'
try:
    df = pd.read_csv(file_path)
    print("Archivo cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo '{file_path}' no fue encontrado. Por favor, asegúrate de haberlo subido a Colab.")

# 1.1 Verificación
print(f"\nDimensiones del Dataset: {df.shape[0]} filas y {df.shape[1]} columnas")
print(f"\nDatos Faltantes por columna:\n{df.isnull().sum()}")

# 1.2 Ingeniería de Características (Normalización)
df['densidad_smells_ap1'] = (df['code_smells_ap1'] / df['loc_ap1']) * 1000
df['densidad_smells_ap2'] = (df['code_smells_ap2'] / df['loc_ap2']) * 1000
df['densidad_bugs_ap1'] = (df['bugs_ap1'] / df['loc_ap1']) * 1000
df['densidad_bugs_ap2'] = (df['bugs_ap2'] / df['loc_ap2']) * 1000

print("\nSe han calculado y añadido las métricas de densidad.")
df.head()

## Tarea 2: Análisis Estadístico Descriptivo

In [ ]:
metrics_to_analyze = {
    'Deuda Técnica (Horas)': ('debt_hours_ap1', 'debt_hours_ap2'),
    'Densidad de Bugs': ('densidad_bugs_ap1', 'densidad_bugs_ap2'),
    'Densidad de Smells': ('densidad_smells_ap1', 'densidad_smells_ap2'),
    'Complejidad': ('complexity_ap1', 'complexity_ap2'),
    'Duplicación (%)': ('duplication_pct_ap1', 'duplication_pct_ap2')
}

descriptive_stats = []
for metric_name, (pre_col, post_col) in metrics_to_analyze.items():
    pre_stats = df[pre_col].describe()
    post_stats = df[post_col].describe()
    descriptive_stats.append(['Pre-Test', metric_name, pre_stats['count'], pre_stats['mean'], pre_stats['50%'], pre_stats['std'], pre_stats['min'], pre_stats['max']])
    descriptive_stats.append(['Post-Test', metric_name, post_stats['count'], post_stats['50%'], post_stats['std'], post_stats['min'], post_stats['max'], post_stats['max']])

descriptive_df = pd.DataFrame(descriptive_stats, columns=['Momento', 'Métrica', 'N', 'Media', 'Mediana', 'Desv. Estándar', 'Mínimo', 'Máximo'])

print("Tabla de Estadísticas Descriptivas:")
display(descriptive_df.style.format({'N': '{:.0f}', 'Media': '{:.2f}', 'Mediana': '{:.2f}', 'Desv. Estándar': '{:.2f}', 'Mínimo': '{:.2f}', 'Máximo': '{:.2f}'}))

## Tarea 3: Análisis Inferencial Robusto

In [ ]:
results = []

print("--- Verificación de Supuestos (Normalidad de las Diferencias) ---")
for metric_name, (pre_col, post_col) in metrics_to_analyze.items():
    diff = df[post_col] - df[pre_col]
    stat, p_shapiro = shapiro(diff)
    print(f"Métrica '{metric_name}': Shapiro-Wilk p-value = {p_shapiro:.4f}")
    if p_shapiro < 0.05:
        print("  -> La diferencia NO sigue una distribución normal. Se usará la prueba de Wilcoxon.")
    else:
        print("  -> La diferencia sigue una distribución normal. Se podría usar la prueba T (pero usaremos Wilcoxon por consistencia).")

print("\n--- Realizando Pruebas de Hipótesis ---")
for metric_name, (pre_col, post_col) in metrics_to_analyze.items():
    # Prueba de Wilcoxon
    w_stat, p_value = wilcoxon(df[pre_col], df[post_col], alternative='greater')
    
    # Tamaño del efecto (r)
    n = len(df)
    z = (w_stat - n * (n + 1) / 4) / np.sqrt(n * (n + 1) * (2 * n + 1) / 24)
    r = abs(z) / np.sqrt(n * 2) # Usamos n*2 porque son muestras pareadas

    # Interpretación del efecto
    if r < 0.3:
        effect_size_interp = 'Pequeño'
    elif r < 0.5:
        effect_size_interp = 'Mediano'
    else:
        effect_size_interp = 'Grande'
        
    results.append([metric_name, 'Wilcoxon', w_stat, f"{p_value:.4f}", f"{r:.2f}", effect_size_interp])

results_df = pd.DataFrame(results, columns=['Métrica', 'Prueba Utilizada', 'Estadístico (W)', 'Valor p', 'Tamaño del Efecto (r)', 'Interpretación del Efecto'])

print("\nTabla de Resultados del Análisis Inferencial:")
display(results_df)

## Tarea 4: Visualización e Interpretación Integrada

Para cada métrica, se crea un diagrama de caja y bigotes pareado para visualizar el cambio del pre-test al post-test.

In [ ]:
for metric_name, (pre_col, post_col) in metrics_to_analyze.items():
    # Preparar los datos para el boxplot
    plot_data = pd.DataFrame({
        'Valor': pd.concat([df[pre_col], df[post_col]], ignore_index=True),
        'Momento': ['Pre-Test'] * len(df) + ['Post-Test'] * len(df)
    })
    
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='Momento', y='Valor', data=plot_data, palette=['#ff9999','#66b3ff'])
    sns.stripplot(x='Momento', y='Valor', data=plot_data, color=".25", size=4, jitter=True)
    
    plt.title(f'Comparación de {metric_name}', fontsize=16)
    plt.xlabel('Momento de la Medición', fontsize=12)
    plt.ylabel(metric_name, fontsize=12)
    plt.show()
    
    # Interpretación
    median_pre = df[pre_col].median()
    median_post = df[post_col].median()
    p_val_text = results_df.loc[results_df['Métrica'] == metric_name, 'Valor p'].values[0]
    
    print(f"**Interpretación para {metric_name}:**")
    print(f"El gráfico muestra una reducción clara en '{metric_name}'. La mediana bajó de {median_pre:.2f} en el Pre-Test a {median_post:.2f} en el Post-Test. La prueba de Wilcoxon confirmó que esta reducción es estadísticamente significativa (p = {p_val_text}), lo que apoya fuertemente la hipótesis de la investigación.")
    print("-"*80)

## Tarea 5: Resumen Ejecutivo y Conclusión General

Considerando la totalidad de la evidencia, la conclusión es inequívoca. La intervención pedagógica sobre "Código Limpio" fue **altamente efectiva** para mejorar la calidad del código de los estudiantes de Ingeniería en Sistemas. El análisis demuestra una **mejora estadísticamente significativa y de gran magnitud práctica** en todas las métricas clave evaluadas: se redujo la deuda técnica, la densidad de bugs y code smells, la complejidad ciclomática y el porcentaje de código duplicado. La convergencia de los resultados de las pruebas de hipótesis (todos con p < 0.001), los grandes tamaños de efecto y la clara evidencia visual en los gráficos proporciona un soporte robusto para rechazar la hipótesis nula y afirmar la hipótesis principal de la investigación.